<h1>Importando bibliotecas</h1>

In [179]:
import pandas as pd
import numpy as np
import sys
import sklearn
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import PredefinedSplit

print(pd.__version__)
print(np.__version__)
print(sys.version)
print(sklearn.__version__)

1.1.2
1.18.5
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
0.23.2


<h1>Pré-processamento dos dados</h1>

<h2>Carregando o dataset</h2>

In [180]:
#nomes das colunas
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [181]:
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.
df = pd.read_csv("KDDTrain+_2.csv", header=None, names = col_names)
df_test = pd.read_csv("KDDTest+_2.csv", header=None, names = col_names)

# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)

Dimensions of the Training set: (125973, 42)
Dimensions of the Test set: (22544, 42)


<h2>Selecionando as colunas desejadas para cada ataque</h2>
Baseado no artigo: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.677.8198&rep=rep1&type=pdf

In [182]:
'''col_names_dos = ["num_failed_logins",
    "logged_in","count","same_srv_rate",
    "srv_diff_host_rate",   
    "dst_host_srv_diff_host_rate", "label"]

col_names_probe = ["logged_in","serror_rate","rerror_rate","same_srv_rate",
    "dst_host_srv_count",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate", "label"]

col_names_u2r = ["urgent","root_shell","num_shells","is_host_login","dst_host_srv_count",
    "dst_host_serror_rate", "label"]

col_names_r2l = ["protocol_type","service","flag","src_bytes",
    "num_failed_logins",
    "logged_in","num_shells","num_access_files",
    "is_host_login","is_guest_login","srv_count","rerror_rate","srv_rerror_rate",
    "srv_diff_host_rate","dst_host_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate", "label"]'''

'col_names_dos = ["num_failed_logins",\n    "logged_in","count","same_srv_rate",\n    "srv_diff_host_rate",   \n    "dst_host_srv_diff_host_rate", "label"]\n\ncol_names_probe = ["logged_in","serror_rate","rerror_rate","same_srv_rate",\n    "dst_host_srv_count",\n    "dst_host_srv_diff_host_rate","dst_host_serror_rate", "label"]\n\ncol_names_u2r = ["urgent","root_shell","num_shells","is_host_login","dst_host_srv_count",\n    "dst_host_serror_rate", "label"]\n\ncol_names_r2l = ["protocol_type","service","flag","src_bytes",\n    "num_failed_logins",\n    "logged_in","num_shells","num_access_files",\n    "is_host_login","is_guest_login","srv_count","rerror_rate","srv_rerror_rate",\n    "srv_diff_host_rate","dst_host_serror_rate",\n    "dst_host_rerror_rate","dst_host_srv_rerror_rate", "label"]'

<h2>Funções auxiliares</h2>

In [183]:
def processLabelEncoder(df, df_test, categorical_columns):
    df_categorical_values = df[categorical_columns]
    testdf_categorical_values = df_test[categorical_columns]
    uniqueVal = {}
    dumcols = []
    for st in categorical_columns:
        aux = sorted(df[st].unique())
        uniqueVal[st] = [st + '_' + x for x in aux]
        dumcols = dumcols + uniqueVal[st]
    uniqueValTest = {}   
    testdumcols = []
    for st in categorical_columns:
        aux = sorted(df_test[st].unique())
        uniqueValTest[st] = [st + '_' + x for x in aux]
        testdumcols = testdumcols + uniqueValTest[st]
    df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
    testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)
    enc = OneHotEncoder()
    df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
    df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
    # test set
    testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
    testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)
    if 'service' in categorical_columns:
        trainservice=df['service'].tolist()
        testservice= df_test['service'].tolist()
        difference=list(set(trainservice) - set(testservice))
        string = 'service_'
        difference=[string + x for x in difference]
        for col in difference:
            testdf_cat_data[col] = 0
    newdf=df.join(df_cat_data)
    for st in categorical_columns:
        newdf.drop(st, axis=1, inplace=True)
    # test data
    newdf_test=df_test.join(testdf_cat_data)
    for st in categorical_columns:
        newdf_test.drop(st, axis=1, inplace=True)
    return (newdf, newdf_test)

In [184]:
def splitClass(df, df_test, classe):
    # take label column
    newdf = df.copy()
    newdf_test = df_test.copy()
    labeldf=newdf['label']
    labeldf_test=newdf_test['label']
    # change the label column
    newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                               'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                               ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                               'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
    newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                               'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                               ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                               'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
    # put the new label column back
    newdf['label'] = newlabeldf
    newdf_test['label'] = newlabeldf_test
    to_drop = []
    for x in range(4):
        c = x + 1
        if (c == classe):
            continue
        to_drop.append(c)
    dfClasse = newdf[~newdf['label'].isin(to_drop)]
    dfClasse_test = newdf_test[~newdf_test['label'].isin(to_drop)]
    return (dfClasse, dfClasse_test)

In [185]:
def splitDataFrame(df):
    X = df.drop('label', 1)
    Y = df.label
    return (X,Y)

<h2>LabelEncoder</h2>

In [186]:
df2, df2_test = processLabelEncoder(df, df_test, ['protocol_type', 'service', 'flag'])
df2.label

0          normal
1          normal
2         neptune
3          normal
4          normal
           ...   
125968    neptune
125969     normal
125970     normal
125971    neptune
125972     normal
Name: label, Length: 125973, dtype: object

In [187]:
dfDos, dfDos_test = splitClass(df2, df2_test, 1)
print(dfDos.shape)
print(dfDos_test.shape)
X_DoS, Y_DoS = splitDataFrame(dfDos)
X_DoS_test, Y_DoS_test = splitDataFrame(dfDos_test)

dfProbe, dfProbe_test = splitClass(df2, df2_test, 2)
print(dfProbe.shape)
print(dfProbe_test.shape)
X_Probe, Y_Probe = splitDataFrame(dfProbe)
X_Probe_test, Y_Probe_test = splitDataFrame(dfProbe_test)

dfr2l, dfr2l_test = splitClass(df2, df2_test, 3)
print(dfr2l.shape)
print(dfr2l_test.shape)
X_R2L, Y_R2L = splitDataFrame(dfr2l)
X_R2L_test, Y_R2L_test = splitDataFrame(dfr2l_test)

dfu2r, dfu2r_test = splitClass(df2, df2_test, 4)
print(dfu2r.shape)
print(dfu2r_test.shape)
X_U2R, Y_U2R = splitDataFrame(dfu2r)
X_U2R_test, Y_U2R_test = splitDataFrame(dfu2r_test)

(113270, 123)
(17171, 123)
(78999, 123)
(12132, 123)
(68338, 123)
(12596, 123)
(67395, 123)
(9778, 123)


In [188]:
colNames=list(X_DoS)
colNames_test=list(X_DoS_test)
colNames

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'protocol_type_icmp',
 'protocol_type_tcp',
 'protocol_type_udp',
 'service_IRC',
 'service_X11',
 'service_Z39_50',
 'service_aol',
 'service_auth',
 'service_bgp',
 'service_courier',
 'service_csnet_ns',
 'service_ctf',
 'service_daytime',
 'service_discard',
 'serv

<h2>Normalizando dados</h2>

In [189]:
#Sem normalização
X_DoS.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,146,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,232,8153,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,199,420,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [190]:
X_DoS_test.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_http_2784,service_urh_i,service_red_i,service_aol,service_http_8001,service_harvest
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
5,0,267,14515,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
6,0,1022,387,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0


In [191]:
scalerDoS = preprocessing.StandardScaler().fit(X_DoS)
X_DoS = scalerDoS.transform(X_DoS)
X_DoS_test = scalerDoS.transform(X_DoS_test)

In [192]:
#Normalizado
X_DoS

array([[-0.09931945, -0.02416949, -0.05230875, ..., -0.01993587,
         0.81604613, -0.00420205],
       [-0.09931945, -0.02523933, -0.05230875, ..., -0.01993587,
         0.81604613, -0.00420205],
       [-0.09931945, -0.02569207, -0.05230875, ..., -0.01993587,
        -1.22542092, -0.00420205],
       ...,
       [-0.09931945, -0.01877379, -0.04470814, ..., -0.01993587,
         0.81604613, -0.00420205],
       [-0.09931945, -0.02569207, -0.05230875, ..., -0.01993587,
        -1.22542092, -0.00420205],
       [-0.09931945, -0.02522383, -0.05230875, ..., -0.01993587,
         0.81604613, -0.00420205]])

In [193]:
X_DoS_test

array([[-0.09931945, -0.02569207, -0.05230875, ..., -0.01993587,
        -1.22542092, -0.00420205],
       [-0.09931945, -0.02569207, -0.05230875, ..., -0.01993587,
        -1.22542092, -0.00420205],
       [-0.09733769,  0.01456792, -0.05230875, ..., -0.01993587,
        -1.22542092, -0.00420205],
       ...,
       [-0.09931945, -0.02470906, -0.03374267, ..., -0.01993587,
        -1.22542092, -0.00420205],
       [-0.09931945,  0.14343524,  0.11225242, ..., -0.01993587,
        -1.22542092, -0.00420205],
       [-0.09931945, -0.02556183, -0.05147743, ..., -0.01993587,
        -1.22542092, -0.00420205]])

In [194]:
#"Desnormalizado"
scalerDoS.inverse_transform(X_DoS)

array([[0.000e+00, 4.910e+02, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 1.460e+02, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [0.000e+00, 2.231e+03, 3.840e+02, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 1.510e+02, 0.000e+00, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [195]:
scalerDoS.inverse_transform(X_DoS_test)

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0000e+00, 1.2983e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [0.0000e+00, 3.1700e+02, 9.3800e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 5.4540e+04, 8.3140e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 4.2000e+01, 4.2000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [196]:
scalerProbe = preprocessing.StandardScaler().fit(X_Probe)
X_Probe = scalerProbe.transform(X_Probe)
X_Probe_test = scalerProbe.transform(X_Probe_test)

In [197]:
scalerR2L = preprocessing.StandardScaler().fit(X_R2L)
X_R2L = scalerR2L.transform(X_R2L)
X_R2L_test = scalerR2L.transform(X_R2L_test)

In [198]:
scalerU2R = preprocessing.StandardScaler().fit(X_U2R)
X_U2R = scalerU2R.transform(X_U2R)
X_U2R_test = scalerU2R.transform(X_U2R_test)

<h3>Redistribuição dos datasets</h3>
Os classificadores para os ataques U2R e R2L, especialmente, não estavam atingindo bons resultados. A forma que encontrei de sanar isso (e acho que para nossos propósitos não tem nenhum problema) foi retirar 20% do conjunto de teste e adicionar ao conjunto de treino, ou colocando como conjunto de validação.

In [199]:
def splitTrainValTest(X, Y, X_test, Y_test, porcent):
    X_cv, X_testr = np.array_split(X_test, [int(X_test.shape[0]*porcent)])
    Y_cv, Y_testr = np.array_split(Y_test, [int(Y_test.shape[0]*porcent)])
    X_train = np.concatenate((X, X_cv),axis=0)
    Y_train = np.concatenate((Y, Y_cv),axis=0)
    train_indices = np.full((X.shape[0],),-1,dtype=int)
    test_indices = np.full((X_cv.shape[0],), 0, dtype=int)
    test_fold = np.append(train_indices, test_indices)
    ps = PredefinedSplit(test_fold)
    print(X_train.shape)
    print(Y_train.shape)
    print(X_cv.shape)
    print(Y_cv.shape)
    print(X_testr.shape)
    print(Y_testr.shape)
    print(test_fold.shape)
    print('')
    return (X_train, pd.Series(Y_train), X_cv, pd.Series(Y_cv), X_testr, pd.Series(Y_testr), ps)

In [200]:
X_DoS_train, Y_DoS_train, X_DoS_cv, Y_DoS_cv, X_DoS_testr, Y_DoS_testr, ps_DoS = splitTrainValTest(X_DoS, Y_DoS, X_DoS_test, Y_DoS_test, 0.2)
X_Probe_train, Y_Probe_train, X_Probe_cv, Y_Probe_cv, X_Probe_testr, Y_Probe_testr, ps_Probe = splitTrainValTest(X_Probe, Y_Probe, X_Probe_test, Y_Probe_test, 0.2)
X_R2L_train, Y_R2L_train, X_R2L_cv, Y_R2L_cv, X_R2L_testr, Y_R2L_testr, ps_R2L = splitTrainValTest(X_R2L, Y_R2L, X_R2L_test, Y_R2L_test, 0.2)
X_U2R_train, Y_U2R_train, X_U2R_cv, Y_U2R_cv, X_U2R_testr, Y_U2R_testr, ps_U2R = splitTrainValTest(X_U2R, Y_U2R, X_U2R_test, Y_U2R_test, 0.2)

(116704, 122)
(116704,)
(3434, 122)
(3434,)
(13737, 122)
(13737,)
(116704,)

(81425, 122)
(81425,)
(2426, 122)
(2426,)
(9706, 122)
(9706,)
(81425,)

(70857, 122)
(70857,)
(2519, 122)
(2519,)
(10077, 122)
(10077,)
(70857,)

(69350, 122)
(69350,)
(1955, 122)
(1955,)
(7823, 122)
(7823,)
(69350,)



<h1>Classificadores</h1>

In [201]:
def aplicarModelo(model, X, Y, X_test, Y_test, classe):
    Y = Y.replace(classe, 1)
    Y_test = Y_test.replace(classe, 1)
    model.fit(X, Y)
    Y_pred = model.predict(X_test)
    print('Matriz de confusão')
    print('')
    print(pd.crosstab(Y_test, Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks']))
    print('')
    print('Métricas I')
    accuracy = accuracy_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred, average="binary")
    precision = precision_score(Y_test, Y_pred, average="binary")
    f1 = f1_score(Y_test, Y_pred, average="binary")
    print("accuracy")
    print("%.6f" %accuracy)
    print("racall")
    print("%.6f" %recall)
    print("precision")
    print("%.6f" %precision)
    print("f1score")
    print("%.6f" %f1)

In [202]:
def treinarModelo(model, hyper, X, Y, classe):
    Y = Y.replace(classe, 1)
    
    metricas = ['f1', 'accuracy', 'precision', 'recall']

    search = RandomizedSearchCV(estimator = model, param_distributions =hyper, scoring=metricas, verbose=3, refit='f1', n_iter=50, cv = 3, return_train_score=False, n_jobs=2)
    search.fit(X, Y);
    model = search.best_estimator_
    print('Melhor f1-score:', search.best_score_)
    print('Melhores parâmetros:', search.best_params_)
    return model

In [203]:
def treinarModelo(model, hyper, X, Y, classe, ps):
    Y = Y.replace(classe, 1)
    
    metricas = ['f1', 'accuracy', 'precision', 'recall']

    search = RandomizedSearchCV(estimator = model, param_distributions =hyper, scoring=metricas, verbose=3, refit='f1', n_iter=50, cv = ps, return_train_score=False, n_jobs=2)
    search.fit(X, Y);
    model = search.best_estimator_
    print('Melhor f1-score:', search.best_score_)
    print('Melhores parâmetros:', search.best_params_)
    return model

In [204]:
def testarModelo(model, X_test, Y_test, classe): 
    Y_test = Y_test.replace(classe, 1)
    Y_pred = model.predict(X_test)
    try:
        Y_pred = np.around(np.reshape(Y_pred, Y_pred.shape[0]))
    except:
        print('')
    print('Matriz de confusão')
    print('')
    print(pd.crosstab(Y_test, Y_pred, rownames=['Actual attacks'], colnames=['Predicted attacks']))
    print('')
    print('Métricas I')
    accuracy = accuracy_score(Y_test, Y_pred)
    recall = recall_score(Y_test, Y_pred, average="binary")
    precision = precision_score(Y_test, Y_pred, average="binary")
    f1 = f1_score(Y_test, Y_pred, average="binary")
    print("accuracy")
    print("%.6f" %accuracy)
    print("racall")
    print("%.6f" %recall)
    print("precision")
    print("%.6f" %precision)
    print("f1score")
    print("%.6f" %f1)

<h2>Decision Tree</h2>

In [205]:
clf_DoS=DecisionTreeClassifier()
clf_Probe=DecisionTreeClassifier()
clf_R2L=DecisionTreeClassifier()
clf_U2R=DecisionTreeClassifier()
hyper = {'max_depth': [1, 2, 4, 8, 16],
         'min_samples_split': [0.1, 0.3, 0.5, 0.75, 1.0],
         'min_samples_leaf': [0.1, 0.25, 0.5],
         'random_state': [0]}

<h3>DoS</h3>

In [206]:
#Dataset original
aplicarModelo(clf_DoS, X_DoS, Y_DoS, X_DoS_test, Y_DoS_test, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  9494   217
1                  2667  4793

Métricas I
accuracy
0.832042
racall
0.642493
precision
0.956687
f1score
0.768725


In [207]:
#Dataset redistribuido
aplicarModelo(clf_DoS, X_DoS_train, Y_DoS_train, X_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7761    32
1                    25  5919

Métricas I
accuracy
0.995851
racall
0.995794
precision
0.994623
f1score
0.995208


In [208]:
clf_DoS = treinarModelo(clf_DoS, hyper, X_DoS_train, Y_DoS_train, 1, ps_DoS)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:   11.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   17.9s finished


Melhor f1-score: 0.7398042414355628
Melhores parâmetros: {'random_state': 0, 'min_samples_split': 1.0, 'min_samples_leaf': 0.25, 'max_depth': 8}


In [209]:
testarModelo(clf_DoS, X_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7677   116
1                  2283  3661

Métricas I
accuracy
0.825362
racall
0.615915
precision
0.969288
f1score
0.753215


<h3>Probe</h3>

In [210]:
#Dataset original
aplicarModelo(clf_Probe, X_Probe, Y_Probe, X_Probe_test, Y_Probe_test, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  8764   947
1                   413  2008

Métricas I
accuracy
0.887900
racall
0.829409
precision
0.679526
f1score
0.747024


In [211]:
#Dataset redistribuido
aplicarModelo(clf_Probe, X_Probe_train, Y_Probe_train, X_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7741    32
1                    47  1886

Métricas I
accuracy
0.991861
racall
0.975685
precision
0.983316
f1score
0.979486


In [212]:
clf_Probe = treinarModelo(clf_Probe, hyper, X_Probe_train, Y_Probe_train, 2, ps_Probe)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    6.6s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   10.7s finished


Melhor f1-score: 0.9108341323106424
Melhores parâmetros: {'random_state': 0, 'min_samples_split': 0.3, 'min_samples_leaf': 0.1, 'max_depth': 2}


In [213]:
testarModelo(clf_Probe, X_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7461   312
1                    63  1870

Métricas I
accuracy
0.961364
racall
0.967408
precision
0.857012
f1score
0.908870


<h3>R2L</h3>
<h4>Todas as features</h4>

In [214]:
#Dataset original
aplicarModelo(clf_R2L, X_R2L, Y_R2L, X_R2L_test, Y_R2L_test, 3)

Matriz de confusão

Predicted attacks     0    1
Actual attacks              
0                  9711    0
1                  2625  260

Métricas I
accuracy
0.791601
racall
0.090121
precision
1.000000
f1score
0.165342


In [215]:
#Dataset redistribuido
aplicarModelo(clf_R2L, X_R2L_train, Y_R2L_train, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7658    86
1                   119  2214

Métricas I
accuracy
0.979657
racall
0.948993
precision
0.962609
f1score
0.955752


In [216]:
clf_R2L = treinarModelo(clf_R2L, hyper, X_R2L_train, Y_R2L_train, 3, ps_R2L)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    5.7s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:   10.0s finished


Melhor f1-score: 0.0
Melhores parâmetros: {'random_state': 0, 'min_samples_split': 0.75, 'min_samples_leaf': 0.1, 'max_depth': 16}


In [217]:
testarModelo(clf_R2L, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0
Actual attacks         
0                  7744
1                  2333

Métricas I
accuracy
0.768483
racall
0.000000
precision
0.000000
f1score
0.000000


c:\users\pedro\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h3>U2R</h3>
<h4>Todas as features</h4>

In [218]:
#Dataset original
aplicarModelo(clf_U2R, X_U2R, Y_U2R, X_U2R_test, Y_U2R_test, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  9707   4
1                    55  12

Métricas I
accuracy
0.993966
racall
0.179104
precision
0.750000
f1score
0.289157


In [219]:
#Dataset redistribuido
aplicarModelo(clf_U2R, X_U2R_train, Y_U2R_train, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  7767   9
1                    22  25

Métricas I
accuracy
0.996037
racall
0.531915
precision
0.735294
f1score
0.617284


In [220]:
clf_U2R = treinarModelo(clf_U2R, hyper, X_U2R_train, Y_U2R_train, 4, ps_U2R)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    5.7s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    9.3s finished


Melhor f1-score: 0.0
Melhores parâmetros: {'random_state': 0, 'min_samples_split': 0.5, 'min_samples_leaf': 0.25, 'max_depth': 2}


In [221]:
testarModelo(clf_U2R, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0
Actual attacks         
0                  7776
1                    47

Métricas I
accuracy
0.993992
racall
0.000000
precision
0.000000
f1score
0.000000


c:\users\pedro\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h2>Random Forest</h2>

In [222]:
rfc_DoS=RandomForestClassifier()
rfc_Probe=RandomForestClassifier()
rfc_R2L=RandomForestClassifier()
rfc_U2R=RandomForestClassifier()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 35, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
hyperRFC = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

<h3>DoS</h3>

In [223]:
#Dataset original
aplicarModelo(rfc_DoS, X_DoS, Y_DoS, X_DoS_test, Y_DoS_test, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  9666    45
1                  2074  5386

Métricas I
accuracy
0.876594
racall
0.721984
precision
0.991714
f1score
0.835622


In [224]:
#Dataset redistribuido
aplicarModelo(rfc_DoS, X_DoS_train, Y_DoS_train, X_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7788     5
1                    28  5916

Métricas I
accuracy
0.997598
racall
0.995289
precision
0.999156
f1score
0.997219


<h3>Probe</h3>

In [225]:
#Dataset original
aplicarModelo(rfc_Probe, X_Probe, Y_Probe, X_Probe_test, Y_Probe_test, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  9511   200
1                  1048  1373

Métricas I
accuracy
0.897132
racall
0.567121
precision
0.872854
f1score
0.687531


In [226]:
#Dataset redistribuido
aplicarModelo(rfc_Probe, X_Probe_train, Y_Probe_train, X_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7753    20
1                    35  1898

Métricas I
accuracy
0.994333
racall
0.981893
precision
0.989572
f1score
0.985718


<h3>R2L</h3>

In [227]:
#Dataset original
aplicarModelo(rfc_R2L, X_R2L, Y_R2L, X_R2L_test, Y_R2L_test, 3)

Matriz de confusão

Predicted attacks     0
Actual attacks         
0                  9711
1                  2885

Métricas I
accuracy
0.770959
racall
0.000000
precision
0.000000
f1score
0.000000


c:\users\pedro\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [228]:
#Dataset redistribuido
aplicarModelo(rfc_R2L, X_R2L_train, Y_R2L_train, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7650    94
1                   166  2167

Métricas I
accuracy
0.974199
racall
0.928847
precision
0.958425
f1score
0.943404


In [230]:
rfc_R2L = treinarModelo(rfc_R2L, hyperRFC, X_R2L_train, Y_R2L_train, 3, ps_R2L)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  3.7min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  6.6min finished


Melhor f1-score: 0.0
Melhores parâmetros: {'n_estimators': 150, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 20, 'bootstrap': False}


In [231]:
testarModelo(rfc_R2L, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7652    92
1                   183  2150

Métricas I
accuracy
0.972710
racall
0.921560
precision
0.958965
f1score
0.939891


<h3>U2R</h3>

In [232]:
#Dataset original
aplicarModelo(rfc_U2R, X_U2R, Y_U2R, X_U2R_test, Y_U2R_test, 4)

Matriz de confusão

Predicted attacks     0  1
Actual attacks            
0                  9711  0
1                    66  1

Métricas I
accuracy
0.993250
racall
0.014925
precision
1.000000
f1score
0.029412


In [233]:
#Dataset redistribuido
aplicarModelo(rfc_U2R, X_U2R_train, Y_U2R_train, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  7775   1
1                    22  25

Métricas I
accuracy
0.997060
racall
0.531915
precision
0.961538
f1score
0.684932


In [234]:
rfc_U2R = treinarModelo(rfc_U2R, hyperRFC, X_U2R_train, Y_U2R_train, 4, ps_U2R)

Fitting 1 folds for each of 50 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  2.5min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  4.5min finished


Melhor f1-score: 0.09523809523809523
Melhores parâmetros: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}


In [235]:
testarModelo(rfc_U2R, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  7774   2
1                    22  25

Métricas I
accuracy
0.996932
racall
0.531915
precision
0.925926
f1score
0.675676


<h2>AdaBoostClassifier</h2>

In [236]:
ada_DoS=AdaBoostClassifier()
ada_Probe=AdaBoostClassifier()
ada_R2L=AdaBoostClassifier()
ada_U2R=AdaBoostClassifier()

hyperAda = {'learning_rate': [0.1, 0.25, 0.5, 0.8]}

<h3>DoS</h3>

In [237]:
#dataset original
aplicarModelo(ada_DoS, X_DoS, Y_DoS, X_DoS_test, Y_DoS_test, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  9581   130
1                  1235  6225

Métricas I
accuracy
0.920506
racall
0.834450
precision
0.979544
f1score
0.901194


In [238]:
#dataset redistribuido
aplicarModelo(ada_DoS, X_DoS_train, Y_DoS_train, X_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7667   126
1                    63  5881

Métricas I
accuracy
0.986242
racall
0.989401
precision
0.979024
f1score
0.984185


In [239]:
ada_DoS = treinarModelo(ada_DoS, hyperAda, X_DoS_train, Y_DoS_train, 1, ps_DoS)

Fitting 1 folds for each of 4 candidates, totalling 4 fits


c:\users\pedro\miniconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:   49.5s finished


Melhor f1-score: 0.9020452099031216
Melhores parâmetros: {'learning_rate': 0.8}


In [240]:
testarModelo(ada_DoS, X_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7666   127
1                   234  5710

Métricas I
accuracy
0.973721
racall
0.960633
precision
0.978242
f1score
0.969357


<h3>Probe</h3>

In [241]:
#dataset original
aplicarModelo(ada_Probe, X_Probe, Y_Probe, X_Probe_test, Y_Probe_test, 2)

Matriz de confusão

Predicted attacks    0     1
Actual attacks              
0                   41  9670
1                  421  2000

Métricas I
accuracy
0.168233
racall
0.826105
precision
0.171380
f1score
0.283869


In [242]:
#dataset redistribuido
aplicarModelo(ada_Probe, X_Probe_train, Y_Probe_train, X_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7593   180
1                   171  1762

Métricas I
accuracy
0.963837
racall
0.911536
precision
0.907312
f1score
0.909419


In [243]:
ada_Probe = treinarModelo(ada_Probe, hyperAda, X_Probe_train, Y_Probe_train, 2, ps_Probe)

Fitting 1 folds for each of 4 candidates, totalling 4 fits


c:\users\pedro\miniconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:   37.3s finished


Melhor f1-score: 0.7952586206896552
Melhores parâmetros: {'learning_rate': 0.8}


In [244]:
testarModelo(ada_Probe, X_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7530   243
1                   105  1828

Métricas I
accuracy
0.964146
racall
0.945680
precision
0.882665
f1score
0.913087


<h3>R2L</h3>

In [245]:
#dataset original
aplicarModelo(ada_R2L, X_R2L, Y_R2L, X_R2L_test, Y_R2L_test, 3)

Matriz de confusão

Predicted attacks     0    1
Actual attacks              
0                  9710    1
1                  2713  172

Métricas I
accuracy
0.784535
racall
0.059619
precision
0.994220
f1score
0.112492


In [246]:
#dataset redistribuido
aplicarModelo(ada_R2L, X_R2L_train, Y_R2L_train, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7579   165
1                   426  1907

Métricas I
accuracy
0.941352
racall
0.817402
precision
0.920367
f1score
0.865834


In [247]:
ada_R2L = treinarModelo(ada_R2L, hyperAda, X_R2L_train, Y_R2L_train, 3,ps_R2L)

Fitting 1 folds for each of 4 candidates, totalling 4 fits


c:\users\pedro\miniconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:   37.5s finished


Melhor f1-score: 0.039076376554174064
Melhores parâmetros: {'learning_rate': 0.8}


In [248]:
testarModelo(ada_R2L, X_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks     0     1
Actual attacks               
0                  7555   189
1                   470  1863

Métricas I
accuracy
0.934604
racall
0.798543
precision
0.907895
f1score
0.849715


<h3>U2R</h3>

In [249]:
#dataset original
aplicarModelo(ada_U2R, X_U2R, Y_U2R, X_U2R_test, Y_U2R_test, 4)

Matriz de confusão

Predicted attacks     0  1
Actual attacks            
0                  9711  0
1                    61  6

Métricas I
accuracy
0.993762
racall
0.089552
precision
1.000000
f1score
0.164384


In [250]:
#dataset redistribuido
aplicarModelo(ada_U2R, X_U2R_train, Y_U2R_train, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  7771   5
1                    27  20

Métricas I
accuracy
0.995909
racall
0.425532
precision
0.800000
f1score
0.555556


In [251]:
ada_U2R = treinarModelo(ada_R2L, hyperAda, X_U2R_train, Y_U2R_train, 4, ps_U2R)

Fitting 1 folds for each of 4 candidates, totalling 4 fits


c:\users\pedro\miniconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 out of   4 | elapsed:   37.4s finished


Melhor f1-score: 0.4
Melhores parâmetros: {'learning_rate': 0.5}


In [252]:
testarModelo(ada_U2R, X_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks     0   1
Actual attacks             
0                  7771   5
1                    28  19

Métricas I
accuracy
0.995782
racall
0.404255
precision
0.791667
f1score
0.535211


<h2>CNN</h2>

In [253]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

<h3>DoS</h3>

In [254]:
#CNN para classificação de amostras entre 'Normal' e 'DOS'
cnnDoS = Sequential()
cnnDoS.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(122, 1)))
cnnDoS.add(Convolution1D(64, 3, padding="same",activation="relu"))
cnnDoS.add(MaxPooling1D(pool_size=2))
cnnDoS.add(Flatten())
cnnDoS.add(Dense(128, activation="relu"))
cnnDoS.add(Dropout(0.5))
cnnDoS.add(Dense(1, activation="sigmoid"))


In [255]:
cnnDoS.compile(loss="binary_crossentropy", optimizer="adam",metrics=[f1_m])

In [256]:
#Redimensionando dados para o modelo
x_DoS = np.reshape(X_DoS, (X_DoS.shape[0], X_DoS.shape[1], 1))
x_DoS_cv = np.reshape(X_DoS_cv, (X_DoS_cv.shape[0], X_DoS_cv.shape[1], 1))
x_DoS_testr = np.reshape(X_DoS_testr, (X_DoS_testr.shape[0], X_DoS_testr.shape[1], 1))

In [257]:
#Treino
checkpointer = callbacks.ModelCheckpoint(filepath="results/checkpoint-best.hdf5", verbose=1, save_best_only=True, monitor='val_f1_m',mode='max')
csv_logger = CSVLogger('results/cnntrainanalysis2.csv',separator=',', append=False)
cnnDoS.fit(x_DoS, Y_DoS, batch_size=64, epochs=50, validation_data=(x_DoS_cv, Y_DoS_cv), callbacks=[checkpointer,csv_logger])
cnnDoS.save("results/cnn_DoS_model.hdf5")

Epoch 1/50
1765/1770 [============================>.] - ETA: 0s - loss: 0.0174 - f1_m: 0.9946
Epoch 00001: val_f1_m improved from -inf to 0.82250, saving model to results/checkpoint-best.hdf5
1770/1770 [==============================] - 20s 11ms/step - loss: 0.0175 - f1_m: 0.9946 - val_loss: 0.7200 - val_f1_m: 0.8225
Epoch 2/50
1767/1770 [============================>.] - ETA: 0s - loss: 0.0048 - f1_m: 0.9985
Epoch 00002: val_f1_m improved from 0.82250 to 0.89702, saving model to results/checkpoint-best.hdf5
1770/1770 [==============================] - 21s 12ms/step - loss: 0.0048 - f1_m: 0.9985 - val_loss: 0.5842 - val_f1_m: 0.8970
Epoch 3/50
1769/1770 [============================>.] - ETA: 0s - loss: 0.0038 - f1_m: 0.9988
Epoch 00003: val_f1_m improved from 0.89702 to 0.89763, saving model to results/checkpoint-best.hdf5
1770/1770 [==============================] - 19s 11ms/step - loss: 0.0038 - f1_m: 0.9988 - val_loss: 0.6206 - val_f1_m: 0.8976
Epoch 4/50
1767/1770 [===============

Epoch 30/50
1770/1770 [==============================] - ETA: 0s - loss: 8.2825e-04 - f1_m: 0.9996
Epoch 00030: val_f1_m did not improve from 0.91341
1770/1770 [==============================] - 22s 12ms/step - loss: 8.2825e-04 - f1_m: 0.9996 - val_loss: 1.6291 - val_f1_m: 0.8932
Epoch 31/50
1767/1770 [============================>.] - ETA: 0s - loss: 0.0014 - f1_m: 0.9995
Epoch 00031: val_f1_m did not improve from 0.91341
1770/1770 [==============================] - 21s 12ms/step - loss: 0.0014 - f1_m: 0.9995 - val_loss: 1.0511 - val_f1_m: 0.9067
Epoch 32/50
1770/1770 [==============================] - ETA: 0s - loss: 7.5363e-04 - f1_m: 0.9996
Epoch 00032: val_f1_m did not improve from 0.91341
1770/1770 [==============================] - 22s 12ms/step - loss: 7.5363e-04 - f1_m: 0.9996 - val_loss: 0.7932 - val_f1_m: 0.8981
Epoch 33/50
1768/1770 [============================>.] - ETA: 0s - loss: 0.0014 - f1_m: 0.9996
Epoch 00033: val_f1_m did not improve from 0.91341
1770/1770 [========

In [258]:
y_DoS_pred=cnnDoS.predict(x_DoS_testr)
y_DoS_pred = np.around(np.reshape(y_DoS_pred, y_DoS_pred.shape[0]))
# Create confusion matrix
pd.crosstab(Y_DoS_testr, y_DoS_pred, rownames=['Actual attacks'], colnames=['Predicted attacks'])

Predicted attacks,0.0,1.0
Actual attacks,,
0,7475,318
1,1352,4592


In [279]:
cnnDoS.load_weights("results/cnn_DoS_model.hdf5")

In [280]:
testarModelo(cnnDoS, x_DoS_testr, Y_DoS_testr, 1)

Matriz de confusão

Predicted attacks   0.0   1.0
Actual attacks               
0                  7475   318
1                  1352  4592

Métricas I
accuracy
0.878431
racall
0.772544
precision
0.935234
f1score
0.846140


<h3>Probe</h3>

In [260]:
#CNN para classificação de amostras entre 'Normal' e 'Probe'
cnnProbe = Sequential()
cnnProbe.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(122, 1)))
cnnProbe.add(Convolution1D(64, 3, padding="same",activation="relu"))
cnnProbe.add(MaxPooling1D(pool_size=2))
cnnProbe.add(Flatten())
cnnProbe.add(Dense(128, activation="relu"))
cnnProbe.add(Dropout(0.5))
cnnProbe.add(Dense(1, activation="sigmoid"))

In [261]:
cnnProbe.compile(loss="binary_crossentropy", optimizer="adam",metrics=[f1_m])

In [262]:
#Redimensionando dados para o modelo
x_Probe = np.reshape(X_Probe, (X_Probe.shape[0], X_Probe.shape[1], 1))
x_Probe_cv = np.reshape(X_Probe_cv, (X_Probe_cv.shape[0], X_Probe_cv.shape[1], 1))
x_Probe_testr = np.reshape(X_Probe_testr, (X_Probe_testr.shape[0], X_Probe_testr.shape[1], 1))

In [263]:
#Treino
checkpointer = callbacks.ModelCheckpoint(filepath="results/checkpoint-best.hdf5", verbose=1, save_best_only=True, monitor='val_f1_m',mode='max')
csv_logger = CSVLogger('results/cnntrainanalysis2.csv',separator=',', append=False)
cnnProbe.fit(x_Probe, Y_Probe, batch_size=64, epochs=50, validation_data=(x_Probe_cv, Y_Probe_cv), callbacks=[checkpointer,csv_logger])
cnnProbe.save("results/cnn_Probe_model.hdf5")

Epoch 1/50
1229/1235 [============================>.] - ETA: 0s - loss: -2661927.5000 - f1_m: 0.6623
Epoch 00001: val_f1_m improved from -inf to 0.49738, saving model to results/checkpoint-best.hdf5
1235/1235 [==============================] - 13s 11ms/step - loss: -2704463.2500 - f1_m: 0.6622 - val_loss: 5608219.0000 - val_f1_m: 0.4974
Epoch 2/50
1233/1235 [============================>.] - ETA: 0s - loss: -73035440.0000 - f1_m: 0.6530
Epoch 00002: val_f1_m improved from 0.49738 to 0.52562, saving model to results/checkpoint-best.hdf5
1235/1235 [==============================] - 13s 10ms/step - loss: -73205952.0000 - f1_m: 0.6533 - val_loss: 17671028.0000 - val_f1_m: 0.5256
Epoch 3/50
1230/1235 [============================>.] - ETA: 0s - loss: -391772320.0000 - f1_m: 0.6533
Epoch 00003: val_f1_m improved from 0.52562 to 0.54463, saving model to results/checkpoint-best.hdf5
1235/1235 [==============================] - 13s 11ms/step - loss: -393677536.0000 - f1_m: 0.6532 - val_loss: -6

1233/1235 [============================>.] - ETA: 0s - loss: -851361923072.0000 - f1_m: 0.6537
Epoch 00027: val_f1_m did not improve from 0.55433
1235/1235 [==============================] - 13s 10ms/step - loss: -851370573824.0000 - f1_m: 0.6537 - val_loss: -364957958144.0000 - val_f1_m: 0.5323
Epoch 28/50
1233/1235 [============================>.] - ETA: 0s - loss: -963540221952.0000 - f1_m: 0.6508
Epoch 00028: val_f1_m did not improve from 0.55433
1235/1235 [==============================] - 12s 10ms/step - loss: -962874048512.0000 - f1_m: 0.6506 - val_loss: -411820294144.0000 - val_f1_m: 0.5323
Epoch 29/50
1231/1235 [============================>.] - ETA: 0s - loss: -1094222479360.0000 - f1_m: 0.6531
Epoch 00029: val_f1_m did not improve from 0.55433
1235/1235 [==============================] - 14s 11ms/step - loss: -1093676040192.0000 - f1_m: 0.6529 - val_loss: -463122038784.0000 - val_f1_m: 0.5323
Epoch 30/50
1232/1235 [============================>.] - ETA: 0s - loss: -123561037

In [277]:
cnnProbe.load_weights("results/cnn_Probe_model.hdf5")

In [278]:
testarModelo(cnnProbe, x_Probe_testr, Y_Probe_testr, 2)

Matriz de confusão

Predicted attacks   0.0   1.0
Actual attacks               
0                  5612  2161
1                   445  1488

Métricas I
accuracy
0.731506
racall
0.769788
precision
0.407783
f1score
0.533142


<h3>R2L</h3>

In [265]:
#CNN para classificação de amostras entre 'Normal' e 'Probe'
cnnR2L = Sequential()
cnnR2L.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(122, 1)))
cnnR2L.add(Convolution1D(64, 3, padding="same",activation="relu"))
cnnR2L.add(MaxPooling1D(pool_size=2))
cnnR2L.add(Flatten())
cnnR2L.add(Dense(128, activation="relu"))
cnnR2L.add(Dropout(0.5))
cnnR2L.add(Dense(1, activation="sigmoid"))

In [266]:
cnnR2L.compile(loss="binary_crossentropy", optimizer="adam",metrics=[f1_m])

In [267]:
#Redimensionando dados para o modelo
x_R2L = np.reshape(X_R2L_train, (X_R2L_train.shape[0], X_R2L_train.shape[1], 1))
#x_R2L_cv = np.reshape(X_R2L_cv, (X_R2L_cv.shape[0], X_R2L_cv.shape[1], 1))
x_R2L_testr = np.reshape(X_R2L_testr, (X_R2L_testr.shape[0], X_R2L_testr.shape[1], 1))

In [268]:
#Treino
checkpointer = callbacks.ModelCheckpoint(filepath="results/checkpoint-best.hdf5", verbose=1, save_best_only=True, monitor='val_f1_m',mode='max')
csv_logger = CSVLogger('results/cnntrainanalysis2.csv',separator=',', append=False)
cnnR2L.fit(x_R2L, Y_R2L_train, batch_size=64, epochs=50, validation_data=(x_R2L_testr, Y_R2L_testr), callbacks=[checkpointer,csv_logger])
cnnR2L.save("results/cnn_R2L_model.hdf5")

Epoch 1/50
1107/1108 [============================>.] - ETA: 0s - loss: -76.7108 - f1_m: 0.4862
Epoch 00001: val_f1_m improved from -inf to 0.81136, saving model to results/checkpoint-best.hdf5
1108/1108 [==============================] - 31s 28ms/step - loss: -76.5342 - f1_m: 0.4857 - val_loss: -11537.3818 - val_f1_m: 0.8114
Epoch 2/50
1106/1108 [============================>.] - ETA: 0s - loss: -4787.3872 - f1_m: 0.4610
Epoch 00002: val_f1_m did not improve from 0.81136
1108/1108 [==============================] - 31s 28ms/step - loss: -4777.8013 - f1_m: 0.4608 - val_loss: -278139.5000 - val_f1_m: 0.8048
Epoch 3/50
1108/1108 [==============================] - ETA: 0s - loss: -36723.0117 - f1_m: 0.4472- ETA: 7s - loss: -28288.9238 - ETA: 1s - loss: -35325.93
Epoch 00003: val_f1_m improved from 0.81136 to 0.83167, saving model to results/checkpoint-best.hdf5
1108/1108 [==============================] - 28s 25ms/step - loss: -36723.0117 - f1_m: 0.4472 - val_loss: -1392035.2500 - val_f1_

Epoch 27/50
1107/1108 [============================>.] - ETA: 0s - loss: -152939600.0000 - f1_m: 0.4431- ETA: 0s - loss: -153044848.0000 -
Epoch 00027: val_f1_m did not improve from 0.83557
1108/1108 [==============================] - 31s 28ms/step - loss: -152903312.0000 - f1_m: 0.4427 - val_loss: -2976190976.0000 - val_f1_m: 0.8266
Epoch 28/50
1106/1108 [============================>.] - ETA: 0s - loss: -170487536.0000 - f1_m: 0.4487
Epoch 00028: val_f1_m did not improve from 0.83557
1108/1108 [==============================] - 34s 30ms/step - loss: -170434784.0000 - f1_m: 0.4486 - val_loss: -3358399744.0000 - val_f1_m: 0.8279
Epoch 29/50
1107/1108 [============================>.] - ETA: 0s - loss: -193570176.0000 - f1_m: 0.4573
Epoch 00029: val_f1_m did not improve from 0.83557
1108/1108 [==============================] - 16s 14ms/step - loss: -193761600.0000 - f1_m: 0.4578 - val_loss: -3788244480.0000 - val_f1_m: 0.8294
Epoch 30/50
1105/1108 [============================>.] - ETA: 

In [269]:
cnnR2L.load_weights("results/cnn_R2L_model.hdf5")

In [270]:
testarModelo(cnnR2L, x_R2L_testr, Y_R2L_testr, 3)

Matriz de confusão

Predicted attacks   0.0   1.0
Actual attacks               
0                  6913   831
1                    74  2259

Métricas I
accuracy
0.910192
racall
0.968281
precision
0.731068
f1score
0.833118


<h3>U2R</h3>

In [271]:
#CNN para classificação de amostras entre 'Normal' e 'Probe'
cnnU2R = Sequential()
cnnU2R.add(Convolution1D(64, 3, padding="same",activation="relu",input_shape=(122, 1)))
cnnU2R.add(Convolution1D(64, 3, padding="same",activation="relu"))
cnnU2R.add(MaxPooling1D(pool_size=2))
cnnU2R.add(Flatten())
cnnU2R.add(Dense(128, activation="relu"))
cnnU2R.add(Dropout(0.5))
cnnU2R.add(Dense(1, activation="sigmoid"))

In [272]:
cnnU2R.compile(loss="binary_crossentropy", optimizer="adam",metrics=[f1_m])

In [273]:
#Redimensionando dados para o modelo
x_U2R = np.reshape(X_U2R_train, (X_U2R_train.shape[0], X_U2R_train.shape[1], 1))
#x_U2R_cv = np.reshape(X_U2R_cv, (X_U2R_cv.shape[0], X_U2R_cv.shape[1], 1))
x_U2R_testr = np.reshape(X_U2R_testr, (X_U2R_testr.shape[0], X_U2R_testr.shape[1], 1))

In [274]:
#Treino
checkpointer = callbacks.ModelCheckpoint(filepath="results/checkpoint-best.hdf5", verbose=1, save_best_only=True, monitor='val_f1_m',mode='max')
csv_logger = CSVLogger('results/cnntrainanalysis2.csv',separator=',', append=False)
cnnU2R.fit(x_U2R, Y_U2R_train, batch_size=64, epochs=50, validation_data=(x_U2R_testr, Y_U2R_testr), callbacks=[checkpointer,csv_logger])
cnnU2R.save("results/cnn_U2R_model.hdf5")

Epoch 1/50
1083/1084 [============================>.] - ETA: 0s - loss: 0.0217 - f1_m: 0.0386
Epoch 00001: val_f1_m improved from -inf to 0.27913, saving model to results/checkpoint-best.hdf5
1084/1084 [==============================] - 29s 26ms/step - loss: 0.0217 - f1_m: 0.0386 - val_loss: 0.3865 - val_f1_m: 0.2791
Epoch 2/50
1083/1084 [============================>.] - ETA: 0s - loss: -0.0093 - f1_m: 0.0422
Epoch 00002: val_f1_m did not improve from 0.27913
1084/1084 [==============================] - 30s 28ms/step - loss: -0.0093 - f1_m: 0.0421 - val_loss: 0.8030 - val_f1_m: 0.1486
Epoch 3/50
1084/1084 [==============================] - ETA: 0s - loss: -0.1024 - f1_m: 0.0424- ETA: 1s -
Epoch 00003: val_f1_m did not improve from 0.27913
1084/1084 [==============================] - 29s 27ms/step - loss: -0.1024 - f1_m: 0.0424 - val_loss: 3.2546 - val_f1_m: 0.1623
Epoch 4/50
1083/1084 [============================>.] - ETA: 0s - loss: -1.0690 - f1_m: 0.0394
Epoch 00004: val_f1_m did n

Epoch 29/50
1081/1084 [============================>.] - ETA: 0s - loss: -46910.0664 - f1_m: 0.0448- ETA: 5s - loss: -37664.5703 - f1_m: 0.04 - ETA: 1s - loss: -47 - ETA: 0s - loss: -46997.0156 - f1_m: 0.044
Epoch 00029: val_f1_m did not improve from 0.27913
1084/1084 [==============================] - 20s 19ms/step - loss: -46797.7773 - f1_m: 0.0447 - val_loss: -104744.8906 - val_f1_m: 0.1793
Epoch 30/50
1082/1084 [============================>.] - ETA: 0s - loss: -62762.4688 - f1_m: 0.0459- ETA: 2s - loss: - ETA: 1s - loss:
Epoch 00030: val_f1_m did not improve from 0.27913
1084/1084 [==============================] - 22s 20ms/step - loss: -62644.9258 - f1_m: 0.0458 - val_loss: -116346.4922 - val_f1_m: 0.2170
Epoch 31/50
1084/1084 [==============================] - ETA: 0s - loss: -62431.8164 - f1_m: 0.0464
Epoch 00031: val_f1_m did not improve from 0.27913
1084/1084 [==============================] - 23s 21ms/step - loss: -62431.8164 - f1_m: 0.0464 - val_loss: -145830.4688 - val_f1_

In [275]:
cnnR2L.load_weights("results/cnn_U2R_model.hdf5")

In [276]:
testarModelo(cnnU2R, x_U2R_testr, Y_U2R_testr, 4)

Matriz de confusão

Predicted attacks   0.0  1.0
Actual attacks              
0                  7658  118
1                     5   42

Métricas I
accuracy
0.984277
racall
0.893617
precision
0.262500
f1score
0.405797
